# Main working file

In [2]:
# imports

## system
import glob
import os

## data
import numpy as np
import pandas as pd



# Preprocessing

## Rewrapping

In [3]:
# Import the function
from util.video_transformation import rewrap_video

%load_ext autoreload
%autoreload 2 

base_path = "data-in"
input_folder = "avi"

for team_folder in os.listdir(base_path):
    if team_folder not in ['09_10', '21_22']:
        continue
    path_videos = os.path.join(base_path, team_folder, input_folder, "*.avi")
    files = glob.glob(path_videos)

    for file in files:
        rewrap_video(file, os.path.join(base_path, team_folder))

data-in\09_10\avi\Camera_pp10_navigator_0_20240703_1008.avi
Rewrapping video
Video was already rewrapped
data-in\09_10\avi\Camera_pp10_navigator_0_20240703_1008_rewrapped.avi
Video already has metadata
data-in\09_10\avi\Camera_pp9_pilot_0_20240703_1008.avi
Video already has metadata
data-in\21_22\avi\Camera_pp21_pilot_0_20240925_1339.avi
Video already has metadata
data-in\21_22\avi\Camera_pp22_navigator_0_20240925_1338.avi
Rewrapping video
Video was already rewrapped
data-in\21_22\avi\Camera_pp22_navigator_0_20240925_1338_rewrapped.avi
Video already has metadata


## Merge audio and video

In [4]:
# from util.video_transformation import merge_audio_video

# video_path = r'C:\Users\bruinj\OneDrive - TNO\Project Pages\Team Metrics\Team\Work\Experiment\data\21_22\analysis\pp21_navigator_reconstructed_video_2.avi'
# audio_path = r'C:\Users\bruinj\OneDrive - TNO\Project Pages\Team Metrics\Team\Work\Experiment\data\21_22\analysis\navigator_21_discussion_phase_1_audio.wav'
# output_path = r'C:\Users\bruinj\OneDrive - TNO\Project Pages\Team Metrics\Team\Work\Experiment\data\21_22\analysis\navigator_21_discussion_phase_1_audio_vid.avi'

# merge_audio_video()

## Trim Video

# Extraction of Action Units (AUs)

In [ ]:
### all in one doc:
import util.feature_extraction as fe
import os

openface_path = r"C:\Users\Erik\Documents\facial-expression-synchrony\OpenFace_2.2.0_win_x64\OpenFace_2.2.0_win_x64\OpenFace_2.2.0_win_x64\FeatureExtraction.exe"
  
data_in = r"data"
data_out = r"data-out"
folder = "au"
phases = [("instructional_video", 1), ("discussion_phase", 2), ("reschu_run", 8)]
for i in range(5,100)[::2]:
    j = i + 1
    if i < 10: i = "0" + str(i)
    if j < 10: j = "0" + str(j)
    pair = f"{i}_{j}"
    print(pair)
    for phase, count in phases:
        for c in range(count):
            nav = f"pp{i}_navigator_{phase}_{c}"
            pil = f"pp{j}_pilot_{phase}_{c}"
            for participant in [nav, pil]:
                input = os.path.join(data_in, participant + "_reconstructed_video.avi")
                output = os.path.join(data_out, pair, folder, participant + ".csv")
                if os.path.exists(input):
                    if os.path.exists(output):
                        print(f"{participant} already processed, skipping")
                        continue
                    else:
                        print(f"{participant} extraction")
                        os.system(f"{openface_path} -f \"{input}\" -aus -of \"{output}\"")
                else:
                    print(f"{participant} avi file does not exist in the input folder")


05_06
pp05_navigator_instructional_video_0 already processed, skipping
pp06_pilot_instructional_video_0 already processed, skipping
pp05_navigator_discussion_phase_0 already processed, skipping
pp06_pilot_discussion_phase_0 already processed, skipping
pp05_navigator_discussion_phase_1 already processed, skipping
pp06_pilot_discussion_phase_1 already processed, skipping
pp05_navigator_reschu_run_0 extraction


In [1]:
from multiprocessing import Pool, cpu_count

cpu_count() - 1

7

In [ ]:
### FOLLOWING OLD STRUCTURE
# from util.feature_extraction import extract_features
# data_in = r'data-in'
# data_out = r'data-out'

# for pair in os.listdir(data_in):
#     if pair == "21_22":
#         continue
#     input_folder = os.path.join(data_in, pair)
#     output_folder = os.path.join(data_out, pair, 'au')
#     for participant in pair.split('_'):
#         # TODO: handle missing data (NaN or recovered)
#         if os.path.exists(input_folder):
#             aus = extract_features(input_folder, participant, output_folder)


pp05_navigator_instructional_video_0.csv has already been processed. Continuing...
pp05_navigator_discussion_phase_0.csv has already been processed. Continuing...
pp05_navigator_discussion_phase_1.csv has already been processed. Continuing...
pp06_pilot_instructional_video_0.csv has already been processed. Continuing...
pp06_pilot_discussion_phase_0.csv has already been processed. Continuing...
pp06_pilot_discussion_phase_1.csv has already been processed. Continuing...
pp07_navigator_instructional_video_0.csv has already been processed. Continuing...
pp07_navigator_discussion_phase_0.csv has already been processed. Continuing...
pp07_navigator_discussion_phase_1.csv has already been processed. Continuing...
pp08_pilot_instructional_video_0.csv has already been processed. Continuing...
pp08_pilot_discussion_phase_0.csv has already been processed. Continuing...
pp08_pilot_discussion_phase_1.csv has already been processed. Continuing...
pp09_navigator_instructional_video_0.csv has already

# Feature selection


In [ ]:
import util.feature_selection as fs

location = r"data-out" 
for pair in os.listdir(location):
    filename = os.path.join(location, pair, "selection")
    os.makedirs(filename, exist_ok=True)

## Facial factors

In [ ]:
names = ['discussion_phase_0', 'discussion_phase_1', 'instructional_video_0']
for pair in os.listdir(location):
    # TODO: make it check which pair the file is from and make two dataframes of that. Then combine
    for file in os.listdir(os.path.join(location, pair, "au")):
        if ".csv" in file: 
            filename = os.path.join(location, pair, "au", file)
            participant, _ = file.split("_",1)
            df = pd.read_csv(filename)
            for name in names:
                if name in file:
                    factors = fs.au_to_factors(df)
                    factors.to_csv(os.path.join(location, pair, "selection", f"{participant}_{name}_factors.csv"), index=False)

## Correlated Component Analysis

In [ ]:
# corrCA takes a df as input. This df should be all files for a pair. 

for pair in os.listdir(location):
    filename = os.path.join(location, pair)
    data = {}
    nav, pil = pair.split("_")
    nav_df = pd.DataFrame()
    pil_df = pd.DataFrame()
    # add all phases to a single dataframe
    for file in os.listdir(os.path.join(filename, "au")):
        if ".csv" in file: 
            df = pd.read_csv(os.path.join(filename, "au", file))
            if ".csv" in file and nav in file:
                nav_df = pd.concat([nav_df, df])
            if ".csv" in file and pil in file:
                pil_df = pd.concat([pil_df, df])
    nav_df, pil_df = fs.make_equal_length(pair, nav_df, pil_df)
    w = fs.corrCA_weights(nav_df, pil_df) #output = pair/pair_corrca_weights.csv
    w.to_csv(os.path.join(filename, f"{pair}_corrca_weights.csv"), index=False)

names = ['discussion_phase_0', 'discussion_phase_1', 'instructional_video_0']
for pair in os.listdir(location):
    for file in os.listdir(os.path.join(location, pair, "au")):
        if ".csv" in file: 
            filename = os.path.join(location, pair, "au", file)
            participant, _ = file.split("_", 1)
            df = pd.read_csv(filename)
            for name in names:
                if name in file:
                    w = pd.read_csv(os.path.join(location, pair, f'{pair}_corrca_weights.csv'))
                    corrca = fs.apply_corrCA_weights(df, w)
                    corrca.to_csv(os.path.join(location, pair, "selection", f"{participant}_{name}_corrca.csv"), index=False)
                    continue


files of pairs in 05_06 do not have the same amount of datapoints - temporary fix has made them equal length
(17, 17)
files of pairs in 07_08 do not have the same amount of datapoints - temporary fix has made them equal length
(17, 17)
(17, 17)


# Correlation measure

In [ ]:
from scipy.stats import pearsonr
import util.feature_selection as fs
import util.correlation_measure as cm

location = r"data-out" 

for pair in os.listdir(location):
    filename = os.path.join(location, pair, "extraction")
    os.makedirs(filename, exist_ok=True)
    

c:\Users\Erik\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytools\persistent_dict.py:52: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


## Pearson


In [ ]:
sets = ['corrca', 'factors']
phases = ['instructional_video_0', 'discussion_phase_0', 'discussion_phase_1']
factors = ['f1', 'f2', 'f3', 'f4', 'f5', 'f6']

# each pair gets a separate file
for pair in os.listdir(location):
    df = pd.DataFrame()
    nav, pil = pair.split("_")
    file = os.path.join(location, pair, 'selection')

    # format:  | factor1 | factor2 | ... | factor 6 | corrca | (per phase)
    for phase in phases:
        correlation = []
        # factors
        nav_factors = pd.read_csv(os.path.join(file, f"pp{nav}_{phase}_factors.csv"))
        pil_factors = pd.read_csv(os.path.join(file, f"pp{pil}_{phase}_factors.csv"))
        nav_factors, pil_factors = fs.make_equal_length(f"{pair}_{phase}", nav_factors, pil_factors)
        for f in factors:
            corr, _ = pearsonr(nav_factors[f].values, pil_factors[f].values)
            correlation.append(corr)

        # corrca
        nav_corrca = pd.read_csv(os.path.join(file, f"pp{nav}_{phase}_corrca.csv"))
        pil_corrca = pd.read_csv(os.path.join(file, f"pp{pil}_{phase}_corrca.csv"))
        nav_corrca, pil_corrca = fs.make_equal_length(f"{pair}_{phase}", nav_corrca, pil_corrca)
        
        corr, _ = pearsonr(nav_corrca['component1'].values, pil_corrca['component1'].values)
        correlation.append(corr)
        
        df[phase] = correlation
    
    # for participant-wise saving
    df.to_csv(os.path.join(location, pair, "extraction", f"{pair}_pearson.csv"))
    
    # for easy access saving
    df.to_csv(os.path.join("results", f"{pair}_pearson.csv"))

files of pairs in 05_06_discussion_phase_0 do not have the same amount of datapoints - temporary fix has made them equal length
files of pairs in 05_06_discussion_phase_0 do not have the same amount of datapoints - temporary fix has made them equal length
files of pairs in 07_08_instructional_video_0 do not have the same amount of datapoints - temporary fix has made them equal length
files of pairs in 07_08_instructional_video_0 do not have the same amount of datapoints - temporary fix has made them equal length
files of pairs in 07_08_discussion_phase_0 do not have the same amount of datapoints - temporary fix has made them equal length
files of pairs in 07_08_discussion_phase_0 do not have the same amount of datapoints - temporary fix has made them equal length
files of pairs in 07_08_discussion_phase_1 do not have the same amount of datapoints - temporary fix has made them equal length
files of pairs in 07_08_discussion_phase_1 do not have the same amount of datapoints - temporary f

## cRQA

In [ ]:

# nav = pd.read_csv(r"C:\Users\Erik\Documents\facial-expression-synchrony\data-out\07_08\selection\pp07_discussion_phase_0_factors.csv")
# pil = pd.read_csv(r"C:\Users\Erik\Documents\facial-expression-synchrony\data-out\07_08\selection\pp08_discussion_phase_0_factors.csv")



# for f in factors:
#     cm.crqa(f, nav[f].values, pil[f].values) 


# nav = pd.read_csv(r"C:\Users\Erik\Documents\facial-expression-synchrony\data-out\07_08\selection\pp07_discussion_phase_0_factors.csv")
# pil = pd.read_csv(r"C:\Users\Erik\Documents\facial-expression-synchrony\data-out\07_08\selection\pp08_discussion_phase_0_factors.csv")

# cm.crqa3(nav.iloc[1:].values, pil.iloc[1:].values)


# nav = pd.read_csv(r"C:\Users\Erik\Documents\facial-expression-synchrony\data-out\07_08\selection\pp07_discussion_phase_0_factors.csv")
# pil = pd.read_csv(r"C:\Users\Erik\Documents\facial-expression-synchrony\data-out\07_08\selection\pp08_discussion_phase_0_factors.csv")

# cm.mimicry(nav.iloc[1:].values, pil.iloc[1:].values)

In [ ]:
nav = pd.read_csv(r"C:\Users\Erik\Documents\facial-expression-synchrony\data-out\07_08\selection\pp07_discussion_phase_0_factors.csv")
pil = pd.read_csv(r"C:\Users\Erik\Documents\facial-expression-synchrony\data-out\07_08\selection\pp08_discussion_phase_0_factors.csv")

cm.crqa("fac",nav['f1'].values, pil['f2'].values)

nav = pd.read_csv(r"C:\Users\Erik\Documents\facial-expression-synchrony\data-out\07_08\selection\pp07_discussion_phase_0_corrca.csv")
pil = pd.read_csv(r"C:\Users\Erik\Documents\facial-expression-synchrony\data-out\07_08\selection\pp08_discussion_phase_0_corrca.csv")

cm.crqa("cor",nav['component1'].values, pil['component1'].values)

[Platform 'Intel(R) OpenCL HD Graphics']
Vendor: Intel(R) Corporation
Version: OpenCL 2.1 
Profile: FULL_PROFILE
Extensions: cl_khr_byte_addressable_store cl_khr_fp16 cl_khr_global_int32_base_atomics cl_khr_global_int32_extended_atomics cl_khr_icd cl_khr_local_int32_base_atomics cl_khr_local_int32_extended_atomics cl_intel_subgroups cl_intel_required_subgroup_size cl_intel_subgroups_short cl_khr_spir cl_intel_accelerator cl_intel_driver_diagnostics cl_khr_priority_hints cl_khr_throttle_hints cl_khr_create_command_queue cl_intel_subgroups_char cl_intel_subgroups_long cl_khr_il_program cl_khr_fp64 cl_khr_subgroups cl_intel_spirv_device_side_avc_motion_estimation cl_intel_spirv_media_block_io cl_intel_spirv_subgroups cl_khr_spirv_no_integer_wrap_decoration cl_intel_unified_shared_memory_preview cl_khr_mipmap_image cl_khr_mipmap_image_writes cl_intel_planar_yuv cl_intel_packed_yuv cl_intel_motion_estimation cl_intel_device_side_avc_motion_estimation cl_intel_advanced_motion_estimation cl_k

(<pyrqa.result.RQAResult at 0x29edef61130>,
 <pyrqa.result.RPResult at 0x29e8003ba40>)